In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
from scipy.interpolate import interp1d
from scipy.integrate import simpson
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from tqdm import tqdm

# Model Skymaps

This notebook produces skymaps of the expected number of neutrino events for different models at different luminosity distances. The expected number of neutrinos is calculated using

$$ N_\nu (\phi,\theta) = \int d E_\nu A_{eff}(E_\nu, \phi, \theta)  \phi_\nu$$

for every point in the sky. 

For different luminosity distances the fluences presented in Ref. 2 and 3. are scaled considering the following. 

The fluence at Earth from a source at a luminosity distance $d_L$ is, Ref. 1., 

$$ E_\nu^2 \phi_\nu(E_\nu)  = \dfrac{1+z}{4 \pi dL^2} \hat{E_\nu}^2 \dfrac{dN}{d \hat{E_\nu}}$$

where the energy at the source is related to the one on earth by $\hat E_\nu = (1+z)E_\nu$.

Denoting the quantities at the original distance with a subscript '0' and ones at the new distance with a subscript 'n' one finds

$$ E_{\nu,n}^2 \phi_{\nu,n}(E_{\nu,n}) = \dfrac{d_{L,0}^2}{d_{L,n}^2} \dfrac{1+z_n}{1+z_0}E_{\nu,0}^2 \phi_{\nu,0}(E_{\nu,0}) \textbar_{E_{\nu,0} = \dfrac{1+z_n}{1+z_0}E_{\nu,n}}  $$

In [2]:
cosmo = FlatLambdaCDM(H0=70 * u.km/u.s/u.Mpc, Om0=0.3)
erg2gev = 624.2            # 1 erg in GeV
pc2cm  = 3.086e18          # 1 pc   in cm

# Energy grid and log-grid 
energies = (np.array([1e6, 1e7, 1e8, 1e9, 1e10]) * 1e9).astype(np.float64) #eV
logE     = np.log(energies) # used for integration

# Style (unchanged)
mpl.rcParams.update({
    'text.usetex':     False,
    'mathtext.fontset':'cm',
    'font.family':     'serif',
    'font.serif':      ['DejaVu Serif'],
    'font.size':       12,
    'axes.labelsize':  14,
})

In [3]:
Aeff_list = []
lon = decs = None
for i in [6,7,8,9,10]:  # log10(E)
    path = f'../effective_areas/skymaps_60FoV/acceptance_{i}.npz'
    with np.load(path) as data:
        Aeff_list.append(data['events'].astype(np.float32))  # smaller dtype is fine
        if lon is None and 'lon' in data and 'decs' in data:
            lon  = data['lon'].astype(np.float32)
            decs = data['decs'].astype(np.float32)

Aeff_all = np.vstack(Aeff_list)  # shape: (5, n_pix)

In [4]:
def make_z_of_dL(max_z=100.0, n=40000):
    '''
    Redshift interpolator makes calculations slightly faster
    '''
    z_grid  = np.linspace(1e-6, max_z, n)
    dL_grid = cosmo.luminosity_distance(z_grid).to_value(u.Mpc)
    return interp1d(dL_grid, z_grid, kind='cubic', bounds_error=False, fill_value='extrapolate')

z_of_dL = make_z_of_dL()

Note that papers use a different (include papers) use different references for each 

In [5]:
def make_fluence_interp(file_path, delimiter=None):
    arr = np.genfromtxt(file_path, delimiter=delimiter) if delimiter else np.genfromtxt(file_path)
    E_native_eV = arr[:,0] * 1e9        # GeV -> eV
    flux_native = arr[:,1]              # in erg^-1 cm^-2 for files here check file convention if changed
    return interp1d(E_native_eV, flux_native, fill_value='extrapolate')

def fluence_at_E(fluence_interp):
    """Base fluence vector on 'energies' (same for all distances, before scaling)."""
    return fluence_interp(energies)  # shape (5,)


# vectorized energy integral over ALL pixels
def expected_events_map(fluence_E_tau, Aeff_stack, energies, gamma=2.0):
    ''' ∫ (fluence * E^{-gamma}) * Aeff dE  ==  ∫ (fluence * E^{1-gamma}) * Aeff d(lnE) '''
    integrand = (fluence_E_tau * energies**(1.0 - gamma))[:, None] * Aeff_stack
    return simpson(integrand, x=np.log(energies), axis=0)

#Model runners 

def scale_vector(dL0_Mpc, dL_Mpc_array, is_all_flavor, z0=None):
    # z0 for reference distance:
    if z0 is None:
        z0 = z_of_dL(dL0_Mpc)
    znew = z_of_dL(dL_Mpc_array)
    factor = (dL0_Mpc**2 / (dL_Mpc_array**2)) * ((1.0 + znew) / (1.0 + z0))
    if not is_all_flavor:
        factor *= 3.0  # go from one flavor template to all, matching your original
    return factor, znew

def run_model(
    dL_array,
    flux_file,
    dL0_Mpc,
    already_all_flavor=False,
    delimiter=None,
    title='',
    output_folder='out',
    do_plot=False,
    save_npz=False,
    progress=True,          # NEW: show a local tqdm bar
    pbar=None,              # NEW: alternatively, update an external/global bar
    tqdm_desc=None          # NEW: custom label for the bar
):
    os.makedirs(output_folder, exist_ok=True)

    flu_interp = make_fluence_interp(flux_file, delimiter=delimiter)
    z0 = z_of_dL(dL0_Mpc)

    dL_array = np.atleast_1d(dL_array).astype(np.float64)

    # Choose iterator: local tqdm, external pbar, or plain loop
    if pbar is not None:
        iterator = dL_array  # we’ll update pbar manually
    elif progress:
        iterator = tqdm(dL_array,
                        desc=tqdm_desc or title or os.path.basename(output_folder),
                        unit='Mpc',
                        dynamic_ncols=True,
                        leave=False)
    else:
        iterator = dL_array

    for dL in iterator:
        # Get both the scale factor and z_new for this distance
        sc_arr, znew_arr = scale_vector(dL0_Mpc, np.array([dL]), already_all_flavor, z0=z0)
        sc   = sc_arr[0]
        znew = znew_arr[0]

        # --- REDSHIFTED ENERGY SAMPLING ---
        # Evaluate the input (Earth-frame-from-template) fluence at E_src = [(1+znew)/(1+z0)] * E_obs
        shift_ratio = (1.0 + znew) / (1.0 + z0)
        base_flu = flu_interp(energies * shift_ratio)  # same shape as 'energies'

        # Continue with your amplitude scaling and unit conversions
        fluE = sc * erg2gev * base_flu * 1e9
        # tau-only
        fluE_tau = fluE / 3.0
        fluE_tau = np.clip(fluE_tau, 0, None)
        
        events_flat = expected_events_map(fluE_tau, Aeff_all, energies, gamma=2.0)

        if do_plot:
            fig, ax = plt.subplots(figsize=(8, 4.5), subplot_kw=dict(projection='mollweide'))
            sca = ax.scatter(lon, decs, c=events_flat, s=4, cmap='inferno')
            cb = fig.colorbar(sca, ax=ax, orientation='horizontal', pad=0.05)
            cb.set_label(r'$N_\nu$')
            ax.grid(True)
            ax.set_title(title + fr'  (d$_L$ ≈ {dL:.1f} Mpc)')
            fig.savefig(os.path.join(output_folder, f'skymap_{dL:.3f}.png'), dpi=150)
            plt.close(fig)

        if save_npz:
            np.savez(os.path.join(output_folder, f'events_{dL:.3f}.npz'),
                     lon=lon, decs=decs, events=events_flat)

        if pbar is not None:
            pbar.update(1)

    return events_flat

In [6]:
dL_s   = np.logspace(0, 3, 15)
events_s = run_model(
    dL_array=dL_s,
    flux_file='files/extended_emission_murase.csv',
    dL0_Mpc=300.0,
    already_all_flavor=False,       # multiplied by 3 here
    delimiter=',',
    title='sGRB Neutrino Event Expectations',
    output_folder='skymaps_expected_events_sGRBs',
    do_plot=True,                  
    save_npz=True, 
    progress=True
)

In [7]:
dL_p = np.logspace(2, 6, 15)
events_p = run_model(dL_p, 'files/Photosphere_GRB.txt', 718.38,
          already_all_flavor=True, delimiter = '	',
          title='HLGRB (Photosphere)', output_folder='skymaps_expected_events_Photosphere_change_dL', do_plot = True, save_npz = True, 
          progress=True)

In [8]:
dL_i = np.logspace(2, 6, 15)
events_i = run_model(dL_i, 'files/Int_Shock_GRB.csv', 718.38,
          already_all_flavor=True, delimiter=',',
          title='HLGRB (Internal Shock)', output_folder='skymaps_expected_events_IShock_change_dL', do_plot = True, save_npz = True,
          progress=True)

# References

1. Shigeo S. Kimura (2023). *Chapter 9: Neutrinos from Gamma-Ray Bursts*. In: Giovanni G. Fazi (ed.). Pages 433–482. doi:[10.1142/9789811282645_0009](https://doi.org/10.1142/9789811282645_0009). arXiv:[2202.06480](https://arxiv.org/abs/2202.06480).
2. Shigeo S. Kimura, Kohta Murase, Peter Mészáros, Kenta Kiuchi (2017). *High-Energy Neutrino Emission from Short Gamma-Ray Bursts: Prospects for Coincident Detection with Gravitational Waves*. **Astrophys. J. Lett. 848 (1) L4**. doi:[10.3847/2041-8213/aa8d14](https://doi.org/10.3847/2041-8213/aa8d14). arXiv:[1708.07075](https://arxiv.org/abs/1708.07075).
3. Wenkang Lian, Shunke Ai, He Gao (2024). *Prospect of Gamma-Ray Burst Neutrino Detection with Enhanced Neutrino Detectors*.  **Astrophys. J. Lett. 987 79**. doi[10.3847/1538-4357/add1db](https://iopscience.iop.org/article/10.3847/1538-4357/add1db). arXiv:[2412.16868](https://arxiv.org/abs/2412.16868). 